### Step 1: Connect to WRDS

In [9]:
#!pip install wrds
import wrds
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)

In [10]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [11]:
conn = wrds.Connection(wrds_username='liangqingwen')

Loading library list...
Done


In [12]:
conn.describe_table(library='crsp', table='msf')

Approximately 5037353 rows in crsp.msf.


,name,nullable,type,comment
0,cusip,True,VARCHAR(8),CUSIP Header
1,permno,True,INTEGER,PERMNO
2,permco,True,INTEGER,PERMCO
3,issuno,True,INTEGER,Nasdaq Issue Number
4,hexcd,True,SMALLINT,Exchange Code Header
5,hsiccd,True,INTEGER,Standard Industrial Classification Code
6,date,True,DATE,Date of Observation
7,bidlo,True,"NUMERIC(11, 5)",Bid or Low Price
8,askhi,True,"NUMERIC(11, 5)",Ask or High Price
9,prc,True,"NUMERIC(11, 5)",Price or Bid/Ask Average


### Step 2: Get SP500 Index Membership from CRSP
- I opt for the monthly frequency of the data, but one can choose to work with crsp.dsp500list if more precise date range is needed.

In [13]:
# this list starts from 1/1/2008
sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret, b.vol, b.prc, b.spread,
                        b.shrout, b.retx, b.bidlo, b.askhi
                        from crsp.msp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2008'
                        order by a.permno ;
                        """, date_cols=['start', 'ending', 'date'])
sp500.head(10)

,permno,start,ending,date,ret,vol,prc,spread,shrout,retx,bidlo,askhi
0,10078,1992-08-20,2010-01-28,2008-01-31,-0.034749,3048974.0,17.50,None,792771.0,-0.034749,14.820,17.55
1,10078,1992-08-20,2010-01-28,2008-02-29,-0.062857,2265659.0,16.40,None,792771.0,-0.062857,16.330,17.70
2,10078,1992-08-20,2010-01-28,2008-03-31,-0.053049,2367471.0,15.53,None,782000.0,-0.053049,15.530,16.96
3,10078,1992-08-20,2010-01-28,2008-04-30,0.008371,2189775.0,15.66,None,781784.0,0.008371,14.880,16.00
4,10078,1992-08-20,2010-01-28,2008-05-30,-0.173052,5221244.0,12.95,None,781784.0,-0.173052,12.638,16.33
5,10078,1992-08-20,2010-01-28,2008-06-30,-0.159846,3411786.0,10.88,None,752000.0,-0.159846,10.880,12.94
6,10078,1992-08-20,2010-01-28,2008-07-31,-0.022978,4448763.0,10.63,None,752000.0,-0.022978,8.780,10.79
7,10078,1992-08-20,2010-01-28,2008-08-29,-0.153340,3756307.0,9.00,None,752953.0,-0.153340,8.860,10.86
8,10078,1992-08-20,2010-01-28,2008-09-30,-0.155556,3470283.0,7.60,None,739000.0,-0.155556,6.750,9.39
9,10078,1992-08-20,2010-01-28,2008-10-31,-0.394737,3887882.0,4.60,None,738585.0,-0.394737,4.440,7.49


### Step 3: Add Other Company Identifiers from CRSP.MSENAMES
- This step aims to add TICKER, SHRCD, EXCHCD and etc. 

In [14]:
# Add Other Descriptive Variables

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt,naics, 
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames as a
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
# only keep the most recent company name 
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))
mse.head()

,comnam,ncusip,namedt,nameendt,naics,permno,shrcd,exchcd,hsiccd,ticker
0,OPTIMUM MANUFACTURING INC,68391610,1986-01-07,1986-12-03,None,10000,10,3,3990,OMFGA
1,OPTIMUM MANUFACTURING INC,68391610,1986-12-04,1987-03-09,None,10000,10,3,3990,OMFGA
2,OPTIMUM MANUFACTURING INC,68391610,1987-03-10,1987-06-11,None,10000,10,3,3990,OMFGA
3,GREAT FALLS GAS CO,39040610,1986-01-09,1993-11-21,None,10001,11,3,4925,GFGC
4,ENERGY WEST INC,29274A10,1993-11-22,2004-06-09,None,10001,11,3,4925,EWST


In [15]:
# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions, to ensure the name is valid during on a specific date
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]
sp500_full.head(10)

,permno,start,ending,date,ret,vol,prc,spread,shrout,retx,bidlo,askhi,comnam,ncusip,namedt,nameendt,naics,shrcd,exchcd,hsiccd,ticker
4,10078,1992-08-20,2010-01-28,2008-01-31,-0.034749,3048974.0,17.50,None,792771.0,-0.034749,14.820,17.55,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA
9,10078,1992-08-20,2010-01-28,2008-02-29,-0.062857,2265659.0,16.40,None,792771.0,-0.062857,16.330,17.70,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA
14,10078,1992-08-20,2010-01-28,2008-03-31,-0.053049,2367471.0,15.53,None,782000.0,-0.053049,15.530,16.96,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA
19,10078,1992-08-20,2010-01-28,2008-04-30,0.008371,2189775.0,15.66,None,781784.0,0.008371,14.880,16.00,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA
24,10078,1992-08-20,2010-01-28,2008-05-30,-0.173052,5221244.0,12.95,None,781784.0,-0.173052,12.638,16.33,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA
29,10078,1992-08-20,2010-01-28,2008-06-30,-0.159846,3411786.0,10.88,None,752000.0,-0.159846,10.880,12.94,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA
34,10078,1992-08-20,2010-01-28,2008-07-31,-0.022978,4448763.0,10.63,None,752000.0,-0.022978,8.780,10.79,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA
39,10078,1992-08-20,2010-01-28,2008-08-29,-0.153340,3756307.0,9.00,None,752953.0,-0.153340,8.860,10.86,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA
44,10078,1992-08-20,2010-01-28,2008-09-30,-0.155556,3470283.0,7.60,None,739000.0,-0.155556,6.750,9.39,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA
49,10078,1992-08-20,2010-01-28,2008-10-31,-0.394737,3887882.0,4.60,None,738585.0,-0.394737,4.440,7.49,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA


### Step 4: Add Compustat Identifiers
- Link with Compustat's GVKEY and IID if need to work with fundamental data
- Linkage is done through crsp.ccmxpf_linktable

In [16]:
# Linking with Compustat through CCM

ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

In [17]:
ccm.sample(10)

,gvkey,iid,permno,linktype,linkprim,linkdt,linkenddt
14878,017076,01,10297.0,LC,P,1986-05-27,2017-01-31 00:00:00.000000
32159,127378,01,87489.0,LC,P,1999-12-10,2004-07-30 00:00:00.000000
14442,016481,01,75777.0,LC,C,1989-06-13,1993-04-07 00:00:00.000000
20760,029455,01,80081.0,LU,P,1993-12-21,2018-04-30 00:00:00.000000
15827,019529,01,14425.0,LC,P,2014-01-15,2018-10-31 00:00:00.000000
37714,247881,90,13926.0,LC,P,2013-05-02,2021-05-28 00:00:00.000000
27678,043545,01,24525.0,LS,P,2023-11-02,2024-11-16 10:21:17.065575
31540,119297,01,90046.0,LC,P,2004-02-17,2008-07-03 00:00:00.000000
10805,011665,01,93252.0,LU,C,1985-04-30,1993-01-19 00:00:00.000000
16433,021158,01,76116.0,LC,C,1990-03-27,1999-01-29 00:00:00.000000


In [18]:
# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]
sp500ccm.head(10)
#sp500ccm.sample(5)

,permno,start,ending,date,ret,vol,prc,spread,shrout,retx,bidlo,askhi,comnam,ncusip,namedt,nameendt,naics,shrcd,exchcd,hsiccd,ticker,gvkey,iid,linktype,linkprim,linkdt,linkenddt
0,10078,1992-08-20,2010-01-28,2008-01-31,-0.034749,3048974.0,17.50,None,792771.0,-0.034749,14.820,17.55,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA,012136,01,LC,P,1986-03-04,2010-01-29
1,10078,1992-08-20,2010-01-28,2008-02-29,-0.062857,2265659.0,16.40,None,792771.0,-0.062857,16.330,17.70,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA,012136,01,LC,P,1986-03-04,2010-01-29
2,10078,1992-08-20,2010-01-28,2008-03-31,-0.053049,2367471.0,15.53,None,782000.0,-0.053049,15.530,16.96,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA,012136,01,LC,P,1986-03-04,2010-01-29
3,10078,1992-08-20,2010-01-28,2008-04-30,0.008371,2189775.0,15.66,None,781784.0,0.008371,14.880,16.00,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA,012136,01,LC,P,1986-03-04,2010-01-29
4,10078,1992-08-20,2010-01-28,2008-05-30,-0.173052,5221244.0,12.95,None,781784.0,-0.173052,12.638,16.33,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA,012136,01,LC,P,1986-03-04,2010-01-29
5,10078,1992-08-20,2010-01-28,2008-06-30,-0.159846,3411786.0,10.88,None,752000.0,-0.159846,10.880,12.94,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA,012136,01,LC,P,1986-03-04,2010-01-29
6,10078,1992-08-20,2010-01-28,2008-07-31,-0.022978,4448763.0,10.63,None,752000.0,-0.022978,8.780,10.79,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA,012136,01,LC,P,1986-03-04,2010-01-29
7,10078,1992-08-20,2010-01-28,2008-08-29,-0.153340,3756307.0,9.00,None,752953.0,-0.153340,8.860,10.86,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA,012136,01,LC,P,1986-03-04,2010-01-29
8,10078,1992-08-20,2010-01-28,2008-09-30,-0.155556,3470283.0,7.60,None,739000.0,-0.155556,6.750,9.39,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA,012136,01,LC,P,1986-03-04,2010-01-29
9,10078,1992-08-20,2010-01-28,2008-10-31,-0.394737,3887882.0,4.60,None,738585.0,-0.394737,4.440,7.49,SUN MICROSYSTEMS INC,86681020,2007-12-11,2010-01-26,334111,11,3,3571,JAVA,012136,01,LC,P,1986-03-04,2010-01-29


In [19]:
# Rearrange columns for final output

sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', \
                                  'linktype', 'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip', 'shrcd', 'exchcd', 'naics','hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret','vol', 'prc', 'spread', 'shrout',	'retx',	'bidlo', 'askhi']]
sp500ccm.head(10)

,date,permno,comnam,ncusip,shrcd,exchcd,naics,hsiccd,ticker,gvkey,iid,start,ending,ret,vol,prc,spread,shrout,retx,bidlo,askhi
0,2008-01-31,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,012136,01,1992-08-20,2010-01-28,-0.034749,3048974.0,17.50,None,792771.0,-0.034749,14.820,17.55
1,2008-02-29,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,012136,01,1992-08-20,2010-01-28,-0.062857,2265659.0,16.40,None,792771.0,-0.062857,16.330,17.70
2,2008-03-31,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,012136,01,1992-08-20,2010-01-28,-0.053049,2367471.0,15.53,None,782000.0,-0.053049,15.530,16.96
3,2008-04-30,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,012136,01,1992-08-20,2010-01-28,0.008371,2189775.0,15.66,None,781784.0,0.008371,14.880,16.00
4,2008-05-30,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,012136,01,1992-08-20,2010-01-28,-0.173052,5221244.0,12.95,None,781784.0,-0.173052,12.638,16.33
5,2008-06-30,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,012136,01,1992-08-20,2010-01-28,-0.159846,3411786.0,10.88,None,752000.0,-0.159846,10.880,12.94
6,2008-07-31,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,012136,01,1992-08-20,2010-01-28,-0.022978,4448763.0,10.63,None,752000.0,-0.022978,8.780,10.79
7,2008-08-29,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,012136,01,1992-08-20,2010-01-28,-0.153340,3756307.0,9.00,None,752953.0,-0.153340,8.860,10.86
8,2008-09-30,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,012136,01,1992-08-20,2010-01-28,-0.155556,3470283.0,7.60,None,739000.0,-0.155556,6.750,9.39
9,2008-10-31,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,012136,01,1992-08-20,2010-01-28,-0.394737,3887882.0,4.60,None,738585.0,-0.394737,4.440,7.49


In [20]:
#count the list of s&p every date
cnt = sp500ccm.groupby(['date'])['permno'].count().reset_index().rename(columns={'permno':'npermno'})
cnt.sample(10)

,date,npermno
129,2018-10-31,501
92,2015-09-30,501
16,2009-05-29,501
81,2014-10-31,501
156,2021-01-29,500
147,2020-04-30,501
161,2021-06-30,500
89,2015-06-30,500
137,2019-06-28,501
11,2008-12-31,501


In [21]:
sp500ccm.sample(10)

,date,permno,comnam,ncusip,shrcd,exchcd,naics,hsiccd,ticker,gvkey,iid,start,ending,ret,vol,prc,spread,shrout,retx,bidlo,askhi
54773,2011-12-30,45241,FOREST LABS INC,34583810,11,1,325412,2834,FRX,004843,01,2000-11-22,2014-06-30,0.010013,387512.0,30.26,None,267153.0,0.010013,28.55,30.45
103532,2016-11-30,87184,RED HAT INC,75657710,11,1,518210,7372,RHT,122841,01,2009-07-27,2019-07-12,0.021433,234726.0,79.11,None,179596.0,0.021433,74.45,80.00
114927,2015-11-30,91233,MASTERCARD INC,57636Q10,11,1,541199,7389,MA,160225,01,2008-07-18,2023-12-29,-0.010809,730981.0,97.92,None,1100234.0,-0.010809,96.86,101.50
739,2010-09-30,10145,HONEYWELL INTERNATIONAL INC,43851610,11,1,336412,3724,HON,001300,01,1925-12-31,2023-12-29,0.124936,1016920.0,43.94,None,772210.0,0.124936,40.88,44.46
48762,2011-03-31,35051,PALL CORP,69642930,11,1,333999,3569,PLL,008304,01,1987-12-31,2015-08-28,0.059787,229246.0,57.61,None,116120.0,0.059787,53.44,57.61
25690,2008-03-31,20415,LEUCADIA NATIONAL CORP,52728810,11,1,321912,6211,LUK,006682,01,2007-08-27,2019-09-25,-0.000884,227141.0,45.22,None,222576.0,-0.000884,43.30,46.89
2274,2011-12-30,11081,DELL INC,24702R10,11,3,334113,3570,DELL,014489,01,1996-09-06,2013-10-28,-0.071701,3257409.0,14.63,None,1796509.0,-0.071701,14.63,15.93
43699,2011-03-31,26403,DISNEY WALT CO,25468710,11,1,713110,4833,DIS,003980,01,1976-07-01,2023-12-29,-0.014861,2132182.0,43.09,None,1899069.0,-0.014861,40.60,44.07
3262,2016-06-30,11552,CELGENE CORP,15102010,11,3,325412,2890,CELG,013599,01,2006-11-06,2019-11-20,-0.065296,1051120.0,98.63,None,775000.0,-0.065296,94.85,108.24
6942,2011-06-30,12140,GOODRICH CORP,38238810,11,1,336414,3761,GR,005229,01,1957-03-01,2012-07-30,0.094054,294107.0,95.50,None,124871.0,0.094054,84.60,95.50


In [22]:
### Add CIKs and Link with SEC Index Files using CIK

names = conn.raw_sql(""" select gvkey, cik, sic, gind, gsubind from comp.names """)
# Merge sp500 constituents table with names table
sp500cik = pd.merge(sp500ccm, names, on='gvkey',  how='left')
sp500cik['gvkey'] = pd.to_numeric(sp500cik['gvkey'], errors='coerce').fillna(0).astype(int)
sp500cik.head()


,date,permno,comnam,ncusip,shrcd,exchcd,naics,hsiccd,ticker,gvkey,iid,start,ending,ret,vol,prc,spread,shrout,retx,bidlo,askhi,cik,sic,gind,gsubind
0,2008-01-31,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,12136,01,1992-08-20,2010-01-28,-0.034749,3048974.0,17.50,None,792771.0,-0.034749,14.820,17.55,0000709519,3571,452020,45202010
1,2008-02-29,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,12136,01,1992-08-20,2010-01-28,-0.062857,2265659.0,16.40,None,792771.0,-0.062857,16.330,17.70,0000709519,3571,452020,45202010
2,2008-03-31,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,12136,01,1992-08-20,2010-01-28,-0.053049,2367471.0,15.53,None,782000.0,-0.053049,15.530,16.96,0000709519,3571,452020,45202010
3,2008-04-30,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,12136,01,1992-08-20,2010-01-28,0.008371,2189775.0,15.66,None,781784.0,0.008371,14.880,16.00,0000709519,3571,452020,45202010
4,2008-05-30,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,12136,01,1992-08-20,2010-01-28,-0.173052,5221244.0,12.95,None,781784.0,-0.173052,12.638,16.33,0000709519,3571,452020,45202010


In [23]:
# enerate year-month
#sp500cik = sp500cik.copy()
sp500cik['ym'] = sp500cik['date'].dt.strftime('%Y-%m')

sp500cik.head()

,date,permno,comnam,ncusip,shrcd,exchcd,naics,hsiccd,ticker,gvkey,iid,start,ending,ret,vol,prc,spread,shrout,retx,bidlo,askhi,cik,sic,gind,gsubind,ym
0,2008-01-31,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,12136,01,1992-08-20,2010-01-28,-0.034749,3048974.0,17.50,None,792771.0,-0.034749,14.820,17.55,0000709519,3571,452020,45202010,2008-01
1,2008-02-29,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,12136,01,1992-08-20,2010-01-28,-0.062857,2265659.0,16.40,None,792771.0,-0.062857,16.330,17.70,0000709519,3571,452020,45202010,2008-02
2,2008-03-31,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,12136,01,1992-08-20,2010-01-28,-0.053049,2367471.0,15.53,None,782000.0,-0.053049,15.530,16.96,0000709519,3571,452020,45202010,2008-03
3,2008-04-30,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,12136,01,1992-08-20,2010-01-28,0.008371,2189775.0,15.66,None,781784.0,0.008371,14.880,16.00,0000709519,3571,452020,45202010,2008-04
4,2008-05-30,10078,SUN MICROSYSTEMS INC,86681020,11,3,334111,3571,JAVA,12136,01,1992-08-20,2010-01-28,-0.173052,5221244.0,12.95,None,781784.0,-0.173052,12.638,16.33,0000709519,3571,452020,45202010,2008-05


In [24]:
#count the list of s&p every date
cnt = sp500cik.groupby(['date'])['permno'].count().reset_index().rename(columns={'permno':'npermno'})
cnt.sample(10)

,date,npermno
101,2016-06-30,501
24,2010-01-29,501
111,2017-04-28,501
44,2011-09-30,501
132,2019-01-31,501
129,2018-10-31,501
164,2021-09-30,500
65,2013-06-28,501
167,2021-12-31,500
147,2020-04-30,501


### Step 5: Read Transcriprts
- Link with Compustat's GVKEY with Company Id
- Merge with transcript table

In [25]:
import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
Ear_call = pd.read_csv('Transcript-WRDS/Ear_calls2019.csv')

# Display the first few rows of the DataFrame
Ear_call.head()

,companyid,keydevid,transcriptid,headline,mostimportantdateutc,keydeveventtypename,companyname,audiolengthsec,isdelayed_flag,transcriptid.1,transcriptcomponentid,componentorder,transcriptcomponenttypeid,transcriptcomponenttypename,transcriptpersonid,transcriptpersonname,speakertypename,word_count,componenttext
0,340577.0,597169932.0,1632539.0,"Motorcar Parts of America, Inc., Q2 2019 Earni...",2019-01-08,Earnings Calls,"Motorcar Parts of America, Inc.",3973.0,0,1632539.0,64297491.0,0,1,Presentation Operator Message,1.0,Operator,Operator,47.0,"Good day, ladies and gentlemen, and welcome to..."
1,340577.0,597169932.0,1632539.0,"Motorcar Parts of America, Inc., Q2 2019 Earni...",2019-01-08,Earnings Calls,"Motorcar Parts of America, Inc.",3973.0,0,1632539.0,64297492.0,1,2,Presenter Speech,57497.0,Gary Maier,Executives,232.0,"Thank you. Thank you, all, for joining us for ..."
2,340577.0,597169932.0,1632539.0,"Motorcar Parts of America, Inc., Q2 2019 Earni...",2019-01-08,Earnings Calls,"Motorcar Parts of America, Inc.",3973.0,0,1632539.0,64297493.0,2,2,Presenter Speech,112511.0,Selwyn Joffe,Executives,773.0,"All right. Thank you so much, Gary. I apprecia..."
3,340577.0,597169932.0,1632539.0,"Motorcar Parts of America, Inc., Q2 2019 Earni...",2019-01-08,Earnings Calls,"Motorcar Parts of America, Inc.",3973.0,0,1632539.0,64297494.0,3,2,Presenter Speech,112510.0,David Lee,Executives,1601.0,"Thank you, Selwyn. To begin, I encourage every..."
4,340577.0,597169932.0,1632539.0,"Motorcar Parts of America, Inc., Q2 2019 Earni...",2019-01-08,Earnings Calls,"Motorcar Parts of America, Inc.",3973.0,0,1632539.0,64297495.0,4,2,Presenter Speech,112511.0,Selwyn Joffe,Executives,280.0,"Thanks, David. We're excited by expanding low-..."


In [26]:
type = Ear_call['speakertypename'].unique()
print(type)

['Operator' 'Executives' 'Analysts' 'Shareholders' 'Attendees']


In [27]:
Ear_call['mostimportantdateutc'] = pd.to_datetime( Ear_call['mostimportantdateutc'])
Ear_call['ym'] =  Ear_call['mostimportantdateutc'].dt.strftime('%Y-%m')
#Ear_call.drop(columns=['B'], inplace=True)
Ear_call.head(2)

,companyid,keydevid,transcriptid,headline,mostimportantdateutc,keydeveventtypename,companyname,audiolengthsec,isdelayed_flag,transcriptid.1,transcriptcomponentid,componentorder,transcriptcomponenttypeid,transcriptcomponenttypename,transcriptpersonid,transcriptpersonname,speakertypename,word_count,componenttext,ym
0,340577.0,597169932.0,1632539.0,"Motorcar Parts of America, Inc., Q2 2019 Earni...",2019-01-08,Earnings Calls,"Motorcar Parts of America, Inc.",3973.0,0,1632539.0,64297491.0,0,1,Presentation Operator Message,1.0,Operator,Operator,47.0,"Good day, ladies and gentlemen, and welcome to...",2019-01
1,340577.0,597169932.0,1632539.0,"Motorcar Parts of America, Inc., Q2 2019 Earni...",2019-01-08,Earnings Calls,"Motorcar Parts of America, Inc.",3973.0,0,1632539.0,64297492.0,1,2,Presenter Speech,57497.0,Gary Maier,Executives,232.0,"Thank you. Thank you, all, for joining us for ...",2019-01


In [28]:
gvkey = pd.read_csv('Transcript-WRDS/gvkey.csv')

# Replace NaN values in 'startdate' with an extremely small date
gvkey['startdate'] = gvkey['startdate'].fillna(pd.to_datetime('1677-09-22'))
gvkey.head()

,companyid,gvkey,startdate,enddate,companyname
0,18507.0,235716,1677-09-22 00:00:00,2024-07-24 19:02:02.807778,2M Invest A/S
1,18511.0,210835,1677-09-22 00:00:00,2024-07-24 19:02:02.807778,3i Group plc
2,18527.0,210418,1677-09-22 00:00:00,2024-07-24 19:02:02.807778,ABB Ltd
3,18671.0,29751,1677-09-22 00:00:00,2024-07-24 19:02:02.807778,Albemarle Corporation
4,18711.0,28349,1677-09-22 00:00:00,2024-07-24 19:02:02.807778,The Allstate Corporation


In [29]:
# add date restrictions 
Ear_call = pd.merge(Ear_call, gvkey, on=['companyid'], how='left')
Ear_call.head()

,companyid,keydevid,transcriptid,headline,mostimportantdateutc,keydeveventtypename,companyname_x,audiolengthsec,isdelayed_flag,transcriptid.1,transcriptcomponentid,componentorder,transcriptcomponenttypeid,transcriptcomponenttypename,transcriptpersonid,transcriptpersonname,speakertypename,word_count,componenttext,ym,gvkey,startdate,enddate,companyname_y
0,340577.0,597169932.0,1632539.0,"Motorcar Parts of America, Inc., Q2 2019 Earni...",2019-01-08,Earnings Calls,"Motorcar Parts of America, Inc.",3973.0,0,1632539.0,64297491.0,0,1,Presentation Operator Message,1.0,Operator,Operator,47.0,"Good day, ladies and gentlemen, and welcome to...",2019-01,29930.0,1677-09-22 00:00:00,2024-07-24 19:02:02.807778,"Motorcar Parts of America, Inc."
1,340577.0,597169932.0,1632539.0,"Motorcar Parts of America, Inc., Q2 2019 Earni...",2019-01-08,Earnings Calls,"Motorcar Parts of America, Inc.",3973.0,0,1632539.0,64297492.0,1,2,Presenter Speech,57497.0,Gary Maier,Executives,232.0,"Thank you. Thank you, all, for joining us for ...",2019-01,29930.0,1677-09-22 00:00:00,2024-07-24 19:02:02.807778,"Motorcar Parts of America, Inc."
2,340577.0,597169932.0,1632539.0,"Motorcar Parts of America, Inc., Q2 2019 Earni...",2019-01-08,Earnings Calls,"Motorcar Parts of America, Inc.",3973.0,0,1632539.0,64297493.0,2,2,Presenter Speech,112511.0,Selwyn Joffe,Executives,773.0,"All right. Thank you so much, Gary. I apprecia...",2019-01,29930.0,1677-09-22 00:00:00,2024-07-24 19:02:02.807778,"Motorcar Parts of America, Inc."
3,340577.0,597169932.0,1632539.0,"Motorcar Parts of America, Inc., Q2 2019 Earni...",2019-01-08,Earnings Calls,"Motorcar Parts of America, Inc.",3973.0,0,1632539.0,64297494.0,3,2,Presenter Speech,112510.0,David Lee,Executives,1601.0,"Thank you, Selwyn. To begin, I encourage every...",2019-01,29930.0,1677-09-22 00:00:00,2024-07-24 19:02:02.807778,"Motorcar Parts of America, Inc."
4,340577.0,597169932.0,1632539.0,"Motorcar Parts of America, Inc., Q2 2019 Earni...",2019-01-08,Earnings Calls,"Motorcar Parts of America, Inc.",3973.0,0,1632539.0,64297495.0,4,2,Presenter Speech,112511.0,Selwyn Joffe,Executives,280.0,"Thanks, David. We're excited by expanding low-...",2019-01,29930.0,1677-09-22 00:00:00,2024-07-24 19:02:02.807778,"Motorcar Parts of America, Inc."


In [30]:
# Ensure that all date columns are in datetime format
Ear_call['startdate'] = pd.to_datetime(Ear_call['startdate'])


# Then set link date bounds
Ear_call = Ear_call.loc[(Ear_call['mostimportantdateutc']>=Ear_call['startdate'])\
                        &(Ear_call['mostimportantdateutc']<=Ear_call['enddate'])]

In [31]:
Ear_call.sample()

,companyid,keydevid,transcriptid,headline,mostimportantdateutc,keydeveventtypename,companyname_x,audiolengthsec,isdelayed_flag,transcriptid.1,transcriptcomponentid,componentorder,transcriptcomponenttypeid,transcriptcomponenttypename,transcriptpersonid,transcriptpersonname,speakertypename,word_count,componenttext,ym,gvkey,startdate,enddate,companyname_y
61154,29002.0,597010620.0,1647662.0,"Gilead Sciences, Inc., Q4 2018 Earnings Call, ...",2019-02-04,Earnings Calls,"Gilead Sciences, Inc.",3976.0,0,1647662.0,64949997.0,11,4,Answer,289293.0,Gregg Alton,Executives,117.0,Sure. This is Gregg Alton. I'll take this ques...,2019-02,24856.0,1677-09-22,2024-07-24 19:02:02.807778,"Gilead Sciences, Inc."


In [32]:
# merge sp500cik with transcript
# Merge DataFrames on 'id' and 'date'
#SP_500 = pd.merge(sp500cik,Ear2023, left_on=['gvkey','ym'], right_on=['gvkey_x','ym'],how='inner')
SP_500 = pd.merge(sp500cik, Ear_call, on=['gvkey','ym'],how='inner')

SP_500.head()

,date,permno,comnam,ncusip,shrcd,exchcd,naics,hsiccd,ticker,gvkey,iid,start,ending,ret,vol,prc,spread,shrout,retx,bidlo,askhi,cik,sic,gind,gsubind,ym,companyid,keydevid,transcriptid,headline,mostimportantdateutc,keydeveventtypename,companyname_x,audiolengthsec,isdelayed_flag,transcriptid.1,transcriptcomponentid,componentorder,transcriptcomponenttypeid,transcriptcomponenttypename,transcriptpersonid,transcriptpersonname,speakertypename,word_count,componenttext,startdate,enddate,companyname_y
0,2019-03-29,10104,ORACLE CORP,68389X10,11,1,511210,7379,ORCL,12142,01,1989-08-03,2023-12-29,0.030309,3706972.0,53.71,None,3417654.0,0.030309,52.04,54.04,0001341439,7372,451030,45103020,2019-03,22247.0,605489888.0,1690659.0,"Oracle Corporation, Q3 2019 Earnings Call, Mar...",2019-03-14,Earnings Calls,Oracle Corporation,2705.0,0,1690659.0,66523222.0,0,1,Presentation Operator Message,1.0,Operator,Operator,24.0,Welcome to Oracle's Third Quarter 2019 Earning...,1677-09-22,2024-07-24 19:02:02.807778,Oracle Corporation
1,2019-03-29,10104,ORACLE CORP,68389X10,11,1,511210,7379,ORCL,12142,01,1989-08-03,2023-12-29,0.030309,3706972.0,53.71,None,3417654.0,0.030309,52.04,54.04,0001341439,7372,451030,45103020,2019-03,22247.0,605489888.0,1690659.0,"Oracle Corporation, Q3 2019 Earnings Call, Mar...",2019-03-14,Earnings Calls,Oracle Corporation,2705.0,0,1690659.0,66523223.0,1,2,Presenter Speech,342663.0,Ken Bond,Executives,237.0,"Thank you, operator. Good afternoon, everyone,...",1677-09-22,2024-07-24 19:02:02.807778,Oracle Corporation
2,2019-03-29,10104,ORACLE CORP,68389X10,11,1,511210,7379,ORCL,12142,01,1989-08-03,2023-12-29,0.030309,3706972.0,53.71,None,3417654.0,0.030309,52.04,54.04,0001341439,7372,451030,45103020,2019-03,22247.0,605489888.0,1690659.0,"Oracle Corporation, Q3 2019 Earnings Call, Mar...",2019-03-14,Earnings Calls,Oracle Corporation,2705.0,0,1690659.0,66523224.0,2,2,Presenter Speech,115766.0,Safra Catz,Executives,771.0,"Thanks, Ken. Good afternoon, everyone. I'll fi...",1677-09-22,2024-07-24 19:02:02.807778,Oracle Corporation
3,2019-03-29,10104,ORACLE CORP,68389X10,11,1,511210,7379,ORCL,12142,01,1989-08-03,2023-12-29,0.030309,3706972.0,53.71,None,3417654.0,0.030309,52.04,54.04,0001341439,7372,451030,45103020,2019-03,22247.0,605489888.0,1690659.0,"Oracle Corporation, Q3 2019 Earnings Call, Mar...",2019-03-14,Earnings Calls,Oracle Corporation,2705.0,0,1690659.0,66523225.0,3,2,Presenter Speech,160882.0,Mark Hurd,Executives,1014.0,"Thanks, Safra. Thanks. Solid quarter for us, f...",1677-09-22,2024-07-24 19:02:02.807778,Oracle Corporation
4,2019-03-29,10104,ORACLE CORP,68389X10,11,1,511210,7379,ORCL,12142,01,1989-08-03,2023-12-29,0.030309,3706972.0,53.71,None,3417654.0,0.030309,52.04,54.04,0001341439,7372,451030,45103020,2019-03,22247.0,605489888.0,1690659.0,"Oracle Corporation, Q3 2019 Earnings Call, Mar...",2019-03-14,Earnings Calls,Oracle Corporation,2705.0,0,1690659.0,66523226.0,4,2,Presenter Speech,115765.0,Lawrence Ellison,Executives,311.0,"Thank you, Mark. Oracle's future rests on 2 st...",1677-09-22,2024-07-24 19:02:02.807778,Oracle Corporation


In [33]:
pd.set_option('display.max_rows', 100,'display.max_columns', 1000,"display.max_colwidth",100,'display.width',100)

SP_500.sample(10)
#datacsv.sample(10)

#sample.loc[sample.index[0:10], ['headline', 'mostimportantdateutc']]


,date,permno,comnam,ncusip,shrcd,exchcd,naics,hsiccd,ticker,gvkey,iid,start,ending,ret,vol,prc,spread,shrout,retx,bidlo,askhi,cik,sic,gind,gsubind,ym,companyid,keydevid,transcriptid,headline,mostimportantdateutc,keydeveventtypename,companyname_x,audiolengthsec,isdelayed_flag,transcriptid.1,transcriptcomponentid,componentorder,transcriptcomponenttypeid,transcriptcomponenttypename,transcriptpersonid,transcriptpersonname,speakertypename,word_count,componenttext,startdate,enddate,companyname_y
102305,2019-07-31,83111,ALEXION PHARMACEUTICALS INC,01535110,11,3,325412,2830,ALXN,62263,01,2012-05-25,2021-07-20,-0.135059,436569.0,113.29,None,224227.0,-0.135059,112.36000,134.08,0000899866,2836,352010,35201010,2019-07,347983.0,628779886.0,1779507.0,"Alexion Pharmaceuticals, Inc., Q2 2019 Earnings Call, Jul 24, 2019",2019-07-24,Earnings Calls,"Alexion Pharmaceuticals, Inc.",3331.0,0,1779507.0,69663092.0,2,2,Presenter Speech,331037.0,Ludwig Hantson,Executives,464.0,"Thank you, Susan, and good morning, everyone. We've made great progress so far this year further...",1677-09-22,2024-07-24 19:02:02.807778,"Alexion Pharmaceuticals, Inc."
121547,2019-12-31,89508,CARMAX INC,14313010,11,1,441120,5521,KMX,64410,01,2010-06-28,2023-12-29,-0.098602,336180.0,87.67,None,163795.0,-0.098602,87.43000,99.05,0001170010,5500,255040,25504050,2019-12,356805.0,646836542.0,1887477.0,"CarMax, Inc., Q3 2020 Earnings Call, Dec 20, 2019",2019-12-20,Earnings Calls,"CarMax, Inc.",2640.0,0,1887477.0,73910453.0,33,3,Question,238660.0,Armintas Sinkevicius,Analysts,52.0,"7.5% same-store sales growth year-over-year, that's quite a strong number. How do we think about...",1677-09-22,2024-07-24 19:02:02.807778,"CarMax, Inc."
43252,2019-10-31,34032,MASCO CORP,57459910,11,1,321918,3432,MAS,7085,01,1981-06-18,2023-12-29,0.112884,633463.0,46.25,None,286073.0,0.109645,40.91000,46.25,0000062996,3430,201020,20102010,2019-10,181478.0,638167558.0,1849581.0,"Masco Corporation, Q3 2019 Earnings Call, Oct 30, 2019",2019-10-30,Earnings Calls,Masco Corporation,3539.0,0,1849581.0,72350016.0,25,3,Question,363036.0,Michael Dahl,Analysts,91.0,"That's helpful. A follow-up question, just shifting gears to paint and the environment there. Ob...",1677-09-22,2024-07-24 19:02:02.807778,Masco Corporation
47285,2019-08-30,39490,APACHE CORP,03741110,11,1,211111,1311,APA,1678,01,1997-07-28,2023-12-29,-0.116708,1264558.0,21.57,None,375959.0,-0.116708,19.93000,24.19,0001841666,1311,101020,10102020,2019-08,169838.0,629966686.0,1796303.0,"Apache Corporation, Q2 2019 Earnings Call, Aug 01, 2019",2019-08-01,Earnings Calls,APA Corporation,3236.0,0,1796303.0,70344495.0,64,4,Answer,304916.0,Stephen Riney,Executives,131.0,"Yes. We're probably not going to shed a whole of details on that. But basically, it's structure....",1677-09-22,2024-07-24 19:02:02.807778,APA Corporation
10469,2019-08-30,13142,CAPRI HOLDINGS LTD,G1890L10,12,1,315220,6719,CPRI,198058,01,2013-11-13,2020-05-11,-0.258781,855219.0,26.38,None,151579.0,-0.258781,25.89000,33.97,0001530721,2300,252030,25203010,2019-08,143971118.0,630719436.0,1816972.0,"Capri Holdings Limited, Q1 2020 Earnings Call, Aug 07, 2019",2019-08-07,Earnings Calls,Capri Holdings Limited,3945.0,0,1816972.0,71118066.0,0,1,Presentation Operator Message,1.0,Operator,Operator,43.0,"Good day, and welcome to the Capri Holdings Limited First Quarter 2020 Earnings Conference Call....",1677-09-22,2024-07-24 19:02:02.807778,Capri Holdings Limited
90009,2019-10-31,79057,CHUBB LTD,H1467J10,12,1,524130,6331,CB,28034,01,2010-07-15,2023-12-29,-0.055872,399964.0,152.42,None,453202.0,-0.055872,148.24001,162.06,0000896159,6331,403010,40301040,2019-10,203028.0,596944293.0,1851250.0,"Chubb Limited, Q3 2019 Earnings Call, Oct 30, 2019",2019-10-30,Earnings Calls,Chubb Limited,3602.0,0,1851250.0,72429027.0,38,4,Answer,95732.0,Evan G. Greenberg,Executives,10.0,Am I making sense? Am I being clear for you?,1677-09-22,2024-07-24 19:02:02.807778,Chubb Limited
98238,2019-01-31,81540,AMERISOURCEBERGEN CORP,0

In [34]:
# Convert date column to datetime if not already
SP_500['ym'] = pd.to_datetime(SP_500['ym'])

# Group by 'date' and calculate the number of unique 'id' values for each date
unique_comp = SP_500.groupby('ym')['gvkey'].nunique()

print(unique_comp)

ym
2019-01-01    197
2019-02-01    262
2019-03-01     34
2019-04-01    253
2019-05-01    210
2019-06-01     25
2019-07-01    289
2019-08-01    178
2019-09-01     17
2019-10-01    325
2019-11-01    140
2019-12-01     25
Name: gvkey, dtype: int64


In [35]:
SP_500=SP_500.drop(columns=['companyname_y','transcriptid.1'])
SP_500 = SP_500.rename(columns={'companyname_x': 'companyname'})

In [37]:
SP_500.to_csv('SP500_2019.csv',index=False)

### interact with Stata by using the API

In [47]:
import pandas as pd
pd.set_option('display.max_columns', None)

df=pd.read_csv('SP500_2020.csv')  #read file

In [38]:
df.sample()

NameError: name 'df' is not defined

In [49]:
event=df[['permno','mostimportantdateutc']]
event.sample(5)

,permno,mostimportantdateutc
82042,77129,2020-08-07
2283,11618,2020-10-13
45942,38762,2020-11-02
37761,26403,2020-08-04
100710,85072,2020-05-27


In [50]:
event=event.drop_duplicates()
event.head(5)

,permno,mostimportantdateutc
0,10104,2020-03-12
31,10104,2020-06-16
55,10104,2020-09-10
82,10104,2020-12-10
112,10107,2020-01-29


In [38]:
event.to_csv('event.txt',sep=' ', index=False)

In [39]:
match=df[['permno','mostimportantdateutc','transcriptid']]
match.drop_duplicates()


,permno,mostimportantdateutc,transcriptid
0,10104,2022-03-10,2522984.0
29,10104,2022-06-13,2588744.0
59,10104,2022-09-12,2645685.0
92,10104,2022-12-12,2708926.0
119,10107,2022-01-25,2477196.0
...,...,...,...
120744,93429,2022-11-04,2702934.0
120777,93436,2022-01-26,2478134.0
120873,93436,2022-04-20,2540576.0
120986,93436,2022-07-20,2601753.0


In [40]:
match=match.reset_index(drop=True)
match.drop_duplicates()
match.to_stata('match.dta')